In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

2024-03-10 18:48:53.264345: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 18:48:53.264474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 18:48:54.048841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 18:48:55.482992: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 18:49:00.880841: W tensorflow/compiler/tf2

In [2]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep= ';')
    required_columns = ['Azimuth', 'Altitude', 'Timestamp']

    if not all (col in df.columns for col in required_columns):
        raise ValueError("Required columns are not present in the DataFrame.")
    
    X = df[['Timestamp','Azimuth','Altitude']].values
    y = df[['Azimuth','Altitude']].values

    return X, y, df

In [3]:
csv_path = 'cleaned/cleaned_data_2021.csv'
X, y, df = load_dataset(csv_path)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 42)

In [5]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

In [6]:
timesteps = 1
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

In [7]:
model = Sequential()
model.add(LSTM(units=1024, activation='relu', input_shape=(timesteps, X_train_scaled.shape[1]), return_sequences=True))
model.add(LSTM(units=512, activation='relu', return_sequences=True))
model.add(LSTM(units=256, activation='relu', return_sequences=True))
model.add(LSTM(units=128, activation='relu', return_sequences=True))
model.add(LSTM(units=64, activation='relu', return_sequences=True))
model.add(LSTM(units=32, activation='relu', return_sequences=True))
model.add(LSTM(units=16, activation='relu', return_sequences=True))
model.add(LSTM(units=8, activation='relu'))
model.add(Dense(units=2))

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])  

history = model.fit(X_train_reshaped, y_train_scaled, epochs=500, batch_size=512, validation_data=(X_test_reshaped, y_test_scaled))

2024-03-10 18:49:14.615369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 18:49:17.254914: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 18:49:17.254984: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 18:49:17.260986: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 18:49:17.261068: I external/local_xla/xla/stream_executor

2024-03-10 18:49:22.192560: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/500


2024-03-10 18:49:32.740623: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fce7876a150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-10 18:49:32.740668: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5
2024-03-10 18:49:33.083535: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-10 18:49:33.466941: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1710071373.734118    1142 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


83/83 [==============================] - 18s 70ms/step - loss: 0.3155 - mae: 0.4947 - val_loss: 0.3095 - val_mae: 0.4881
Epoch 2/500
83/83 [==============================] - 4s 53ms/step - loss: 0.3033 - mae: 0.4822 - val_loss: 0.2970 - val_mae: 0.4752
Epoch 3/500
83/83 [==============================] - 4s 54ms/step - loss: 0.2904 - mae: 0.4688 - val_loss: 0.2836 - val_mae: 0.4611
Epoch 4/500
83/83 [==============================] - 5s 57ms/step - loss: 0.2758 - mae: 0.4532 - val_loss: 0.2661 - val_mae: 0.4423
Epoch 5/500
83/83 [==============================] - 5s 60ms/step - loss: 0.1532 - mae: 0.3096 - val_loss: 0.0555 - val_mae: 0.1905
Epoch 6/500
83/83 [==============================] - 5s 60ms/step - loss: 0.0456 - mae: 0.1735 - val_loss: 0.0397 - val_mae: 0.1639
Epoch 7/500
83/83 [==============================] - 5s 58ms/step - loss: 0.0374 - mae: 0.1605 - val_loss: 0.0371 - val_mae: 0.1595
Epoch 8/500
83/83 [==============================] - 5s 59ms/step - loss: 0.0366 - mae:

In [ ]:
predicted_values_scaled = model.predict(X_test_reshaped)
predicted_values = scaler_y.inverse_transform(predicted_values_scaled)
mse = mean_squared_error(y_test_scaled, predicted_values_scaled)
r2 = r2_score(y_test_scaled, predicted_values_scaled)

accuracy = model.evaluate(X_test_reshaped, y_test_scaled)

3287/3287 [==============================] - 24s 7ms/step - loss: 2.2517e-05 - mae: 0.0038


In [ ]:
#timestamps_test = X_test[:, 0]
#df_results = pd.DataFrame({
#    'Timestamp': timestamps_test,
#    'Actual_Azimuth': y_test[:, 0],
#   'Actual_Altitude': y_test[:, 1],
#    'Predicted_Azimuth': predicted_values[:, 0],
#    'Predicted_Altitude': predicted_values[:, 1]
#})

In [ ]:
#output_csv_path = 'azimuth_altitude_prediction_results.csv'
#df_results.to_csv(output_csv_path, index=False)

In [ ]:
print(f"Mean Squared Error on Test Data: {mse}")
print(f"R-squared on Test Data: {r2}")
print(f"Accuracy on Test Data: {accuracy}")
#print(f"Results saved to: {output_csv_path}")

Mean Squared Error on Test Data: 2.2517023265190713e-05
R-squared on Test Data: 0.9996852128883238
Accuracy on Test Data: [2.2517018805956468e-05, 0.0037750808987766504]
Results saved to: azimuth_altitude_prediction_results.csv


In [ ]:
#model.save('model_save')
#model.save_weights('model_weights_1Years.h5')